<a href="https://www.kaggle.com/code/kapturovalexander/kapturov-s-solution-of-leash-bio-1?scriptVersionId=170520799" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

![](https://www.kaggle.com/competitions/67356/images/header)

### My notebook is enspired by Andrew D. Blevins and his notebook [Leash Tutorial - ECFPs and Random Forest](https://www.kaggle.com/code/andrewdblevins/leash-tutorial-ecfps-and-random-forest)

### I will change all his code in my notebook later after I completely figure out with competition

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv


In [2]:
%%time
!pip install rdkit
!pip install duckdb
print('OK')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 62.9 MB/s eta 0:00:00
OK
CPU times: user 368 ms, sys: 90.6 ms, total: 459 ms
Wall time: 28.7 s


![](https://www.chitkara.edu.in/blogs/wp-content/uploads/2023/03/Medicinal-Chemistry-vs-Pharmaceutics.jpg)

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import duckdb
import optuna
from optuna.samplers import TPESampler
import pickle
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import average_precision_score

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [4]:
%%time
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2min 21s, sys: 7.34 s, total: 2min 29s
Wall time: 43.9 s


In [5]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,76925510,O=C(NC1(C(=O)O)CCC1)OCC1c2ccccc2-c2ccccc21,Cl.NCc1cocn1,Nc1ccc(N2CCOCC2)cn1,O=C(N[Dy])C1(Nc2nc(NCc3cocn3)nc(Nc3ccc(N4CCOCC...,sEH,0
1,232843222,O=C(O)C1CCCN1C(=O)OCC1c2ccccc2-c2ccccc21,NCCc1coc2ccccc12,CN1CCN(C(=O)CCN)CC1,CN1CCN(C(=O)CCNc2nc(NCCc3coc4ccccc34)nc(N3CCCC...,HSA,0
2,51448405,Cc1cc(C)c(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)...,COc1cc(CN)c2ccccc2n1,Cc1c([C@@H]2[C@@H](CN)CC(=O)N2C)cnn1C,COc1cc(CNc2nc(NC[C@H]3CC(=O)N(C)[C@@H]3c3cnn(C...,HSA,0
3,122729299,O=C(N[C@H](Cc1ccc(Cl)cc1)C(=O)O)OCC1c2ccccc2-c...,N#Cc1cccnc1N,Cc1cc2cc(CN)ccc2[nH]1,Cc1cc2cc(CNc3nc(Nc4ncccc4C#N)nc(N[C@H](Cc4ccc(...,HSA,0
4,2391443,C#CC[C@@](C)(NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,COC(CN)CC(N)=O.Cl,Cl.NCC1CCOCC12CCCC2,C#CC[C@@](C)(Nc1nc(NCC(CC(N)=O)OC)nc(NCC2CCOCC...,sEH,0


![](https://previews.123rf.com/images/olegdudko/olegdudko1712/olegdudko171200585/91056161-chemistry-science-formula-and-tablets-medicine-symbol.jpg)

In [6]:
%%time
# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

CPU times: user 1min 30s, sys: 1.18 s, total: 1min 31s
Wall time: 1min 31s


In [7]:
%%time
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

# Create and train the random forest model
rf_model = CatBoostClassifier(iterations=100, random_state=27)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.8f}")

Learning rate set to 0.444451
0:	learn: 0.5727496	total: 105ms	remaining: 10.4s
1:	learn: 0.5242736	total: 147ms	remaining: 7.21s
2:	learn: 0.4975015	total: 190ms	remaining: 6.15s
3:	learn: 0.4734350	total: 234ms	remaining: 5.61s
4:	learn: 0.4597268	total: 277ms	remaining: 5.26s
5:	learn: 0.4476222	total: 316ms	remaining: 4.95s
6:	learn: 0.4392631	total: 355ms	remaining: 4.72s
7:	learn: 0.4294194	total: 395ms	remaining: 4.54s
8:	learn: 0.4173052	total: 436ms	remaining: 4.41s
9:	learn: 0.4061772	total: 484ms	remaining: 4.36s
10:	learn: 0.3990633	total: 527ms	remaining: 4.26s
11:	learn: 0.3939141	total: 567ms	remaining: 4.16s
12:	learn: 0.3828786	total: 610ms	remaining: 4.08s
13:	learn: 0.3782214	total: 649ms	remaining: 3.98s
14:	learn: 0.3744288	total: 686ms	remaining: 3.89s
15:	learn: 0.3678572	total: 726ms	remaining: 3.81s
16:	learn: 0.3647159	total: 763ms	remaining: 3.72s
17:	learn: 0.3579223	total: 810ms	remaining: 3.69s
18:	learn: 0.3532191	total: 848ms	remaining: 3.62s
19:	learn: 

![](https://ars.els-cdn.com/content/image/1-s2.0-S2001037020304967-gr3.jpg)

In [8]:
%%time
# Process the test.parquet file chunk by chunk
test_file = '/kaggle/input/leash-BELKA/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
# Wrap the loop with tqdm to display progress
for df_test in tqdm(pd.read_csv(test_file, chunksize=100_000)):
    
    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities
    probabilities = rf_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))

17it [1:02:23, 220.21s/it]

CPU times: user 1h 2min 1s, sys: 38.6 s, total: 1h 2min 40s
Wall time: 1h 2min 23s


![](https://m.media-amazon.com/images/I/41Xf5xwc25L._SR600%2C315_PIWhiteStrip%2CBottomLeft%2C0%2C35_SCLZZZZZZZ_FMpng_BG255%2C255%2C255.jpg)